# How to transpose a CNN

In [1]:
import numpy as np
import tensorflow as tf

num_outputs = 2
x = tf.placeholder(tf.float32, [1, 4, 4, 1])


# custom init with the seed set to 0 by default
def custom_init(shape, dtype=tf.float32, partition_info=None, seed=0):
    return tf.random_normal(shape, dtype=dtype, seed=seed)


np.random.seed(1)
random_array = np.random.rand(1, 4, 4, 1)
print(random_array)

dense_out = tf.layers.dense(x, num_outputs, kernel_initializer=custom_init)
conv_out = tf.layers.conv2d(x, num_outputs, kernel_size=1, strides=1, kernel_initializer=custom_init)

relu = tf.nn.relu(conv_out)
fc0 = tf.contrib.layers.flatten(relu)

# up-sample
upsample_conv = tf.layers.conv2d_transpose(conv_out, 1, (1, 1), (1, 1), kernel_initializer=custom_init)
upsample_dense = tf.layers.conv2d_transpose(dense_out, 1, (1, 1), (1, 1), kernel_initializer=custom_init)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

a, b = sess.run([dense_out, conv_out], feed_dict={x: random_array})
c, d = sess.run([relu, fc0], feed_dict={x: random_array})

print("\nDense Output                        Conv 1x1 Output")
print(np.c_[a, b])
print("Same output? =", np.allclose(a, b, atol=1.e-5))

result_conv, result_dense = sess.run([upsample_conv, upsample_dense], feed_dict={x: random_array})
print("\nresult_con        result_dense")
print(np.c_[result_dense, result_conv])
print("Same output? =", np.allclose(result_conv, result_dense, atol=1.e-5))

[[[[4.17022005e-01]
   [7.20324493e-01]
   [1.14374817e-04]
   [3.02332573e-01]]

  [[1.46755891e-01]
   [9.23385948e-02]
   [1.86260211e-01]
   [3.45560727e-01]]

  [[3.96767474e-01]
   [5.38816734e-01]
   [4.19194514e-01]
   [6.85219500e-01]]

  [[2.04452250e-01]
   [8.78117436e-01]
   [2.73875932e-02]
   [6.70467510e-01]]]]

Dense Output                        Conv 1x1 Output
[[[[-1.6645750e-01  8.7759745e-01 -1.6645750e-01  8.7759745e-01]
   [-2.8752300e-01  1.5158792e+00 -2.8752300e-01  1.5158792e+00]
   [-4.5653578e-05  2.4069485e-04 -4.5653578e-05  2.4069485e-04]
   [-1.2067835e-01  6.3624054e-01 -1.2067835e-01  6.3624054e-01]]

  [[-5.8578730e-02  3.0883884e-01 -5.8578730e-02  3.0883884e-01]
   [-3.6857653e-02  1.9432096e-01 -3.6857653e-02  1.9432096e-01]
   [-7.4347183e-02  3.9197329e-01 -7.4347183e-02  3.9197329e-01]
   [-1.3793319e-01  7.2721159e-01 -1.3793319e-01  7.2721159e-01]]

  [[-1.5837276e-01  8.3497304e-01 -1.5837276e-01  8.3497304e-01]
   [-2.1507281e-01  1.1339071